# Example of Staging Upload to ImageCalc Table
This notebook goes through an example of uploading a classified planet image, a logfile, and training data to the DSWx calval database. Data files and metadata are uploading to a staging bucket. Then, the database manager will commit these uploads to the database

In [1]:
import geopandas as gpd
import pandas as pd
import boto3
import os
from datetime import datetime
import sys
sys.path.insert(0, './tools/')
from addImageCalc import addImageCalc

In [2]:
editor_name = 'Alexander Handwerger'
uploadDir = '/Users/mbonnema/Documents/OPERA_calval/DSWx/Chip_3_4/'
site_name = '3_4'

In [3]:
classified_image_filename = '3_4_classified_2class_ALH.tif'
log_filename = 'Log notes.docx'
#additional_filename = 'additional_file.txt'

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credenitals are needed

In [4]:
bucket_name = 'opera-calval-database-dswx'

In [5]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### Read Image metadata table
To get the geometry metadata for the classified image, we copy the geometry of the source image from the database since the extents are the same. This geometry could also be generated directly from the classified imagery.

In [6]:
imageTable = gpd.read_file(s3.Object(bucket_name,'image.geojson').get()['Body'])
imagecalcTable = gpd.read_file(s3.Object(bucket_name,'image_calc.geojson').get()['Body'])

In [7]:
search = imageTable[imageTable.site_name == '3_4']
search.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
132,0.0,HLS.S30.T20HLJ.2021294T141049.v2.0,2021-10-21T13:30:31,20211021_133031_75_245a,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/3_4/20211021_133031_75_245a/20211021_1330...,97.954495,3_4,0.035727,"POLYGON ((-64.30053 -33.07480, -64.34944 -33.2..."


In [8]:
planet_image = search.iloc[[0]]

In [9]:
geometry = planet_image.geometry.iloc[0]
now = datetime.now().strftime("%Y%m%d_%H%M%S")


### Enter the required file locations and metadata fields
To upload the classified image, we need to specify its location on the local computer (and the location of auxilary files). We also need to fill in some metadata fields. Both file paths and metadata are specified as dictionaries

In [10]:
filePaths = {
    'image_calc' : uploadDir + classified_image_filename,
    'logfile' : uploadDir + log_filename,
    #additional_file: additional_file_name
}


In [12]:
metaData = {
    'image_name':planet_image.image_name.iloc[0], #str
    'image_calc_name':planet_image.image_name.iloc[0]+'_class', #str 
    'previous_name':None, #str
    'calc_type':'Supervised Classification', #str
    'processing_level':'Intermediate', #str
    'oversight_level':None, #str,
    'calculated_by': editor_name, #str
    'reviewed_by': None, #str
    'public':True, #bool
    'geometry':geometry, #shapely geometry
}
metaData

{'image_name': '20211021_133031_75_245a',
 'image_calc_name': '20211021_133031_75_245a_class',
 'previous_name': None,
 'calc_type': 'Supervised Classification',
 'processing_level': 'Intermediate',
 'oversight_level': None,
 'calculated_by': 'Alexander Handwerger',
 'reviewed_by': None,
 'public': True,
 'geometry': <shapely.geometry.polygon.Polygon at 0x7fd361bb2b10>}

### Stage the image
We use a pre-defined function to upload files and metadata to the staging area. This function takes the file paths and metadata dictionaries, as well as the AWS session object as inputs

    

In [13]:
addImageCalc(filePaths,metaData,session)

Creating geojson table
Uploading geojson table
Uploading files
staging complete
